In [19]:
%load_ext autoreload

In [20]:
%autoreload 2

In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from scipy.spatial.distance import cosine
from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict
from utils.utils import *

In [22]:
METADATA_PATH = "../../resources/Lijst_kranten_final.xlsx"
df_meta = pd.read_excel(METADATA_PATH,sheet_name="Sheet1",index_col=0)

In [23]:
df_meta.columns

Index(['PPN', 'Titel Krant', 'Beginjaar', 'Eindjaar', 'Politek', 'Source',
       'Verspreidingsgebied', 'Plaats van uitgave', 'Provincie', 'Zuil'],
      dtype='object')

In [24]:
FEM_PATH = "../../resources/int_vrouw.xlsx"
fem = pd.read_excel(FEM_PATH,sheet_name='Sheet1',index_col=0)

In [25]:
MALE_PATH = "../../resources/int_man.xlsx"
male = pd.read_excel(MALE_PATH,sheet_name='Sheet1',index_col=0)

In [26]:
male.head()

,Trefwoord,Originele spelling,Woordsoort,Selecteer,monosemy,Betekenis
Woordenboek,,,,,,
WNT,omweg,OMWEG,znw.(m.),0,0.0,"Eigenlijk, in tweeledige toepassing."
WNT,onecht,ONECHT (I),"znw.(v.,m.)",0,0.0,"znw., oudtijds vr., thans m.; het mv. niet in..."
WNT,oom,OOM,znw.(m.),1,1.0,Eigenlijk: een broeder des vaders of der moed...
WNT,oorlam,OORLAM,"znw.(m.,o.)",1,1.0,"Eigenlijk, als persoonsnaam, en dan m."
WNT,oppasser,OPPASSER,znw.(m.),1,1.0,De persoon die het toezicht op iets houdt; oo...


In [27]:
male_words = [w.lower().split()[0] for w in male[male.monosemy==1].Trefwoord.values]
male_words = male_words + [w.lower().split()[0] for w in male[male.monosemy==1]["Originele spelling"].values]
male_words = set(male_words)
female_words = [w.lower().split()[0] for w in fem[fem.monosemy==1].Trefwoord.values]
female_words = female_words + [w.lower().split()[0] for w in fem[fem.monosemy==1]["Originele spelling"].values]
female_words = set(female_words)

In [28]:
print(len(female_words),len(male_words))

340 237


In [29]:
female_words.update({'6lotte','dienstmeisje','actrice', 'adjunct-directrice', 'anneline', 'artiste', 'baanbreekster', 'balletdanseres', 'baronnes',
 'bedelaarster', 'bedriegster', 'begeleidster', 'bertilia', 'beschermelinge', 'beschermster', 'bet-overgrootmoeder', 'bewaarster',
 'bierhuishoudster', 'bijzit', 'bloedverwante', 'blondine', 'boerendochter', 'boerenmeid', 'boerenmeisje', 'bonbonniere',
 'brnid', 'burgerjuffrouw', 'cafehoudster', 'caroline', 'chanteuse', 'clairvoyante', 'clara', 'clarke', 'costuumnaaister',
 'czarina', 'dameshand', 'damo', 'danseres', 'daphne', 'demoeder', 'dienstbode', 'dienstmaagd', 'dienstmeid',
 'dievegge', 'direktrice', 'doesschate', 'domoor', 'dora', 'eischeres', 'eischeresse', 'erfdochter', 'espagnole',
 'gastvrouwen', 'gemeente-vroedvrouw', 'gezellin', 'gezellinnen', 'gezelschapsdame', 'gezelschapsjuffrouw', 'gravin',
 'gretchen', 'griekin', 'grisette', 'groentevrouw', 'groolvorstin', 'groot-vorstin', 'groothertogin', 'grootmama', 'grootmeesteresse',
 'grootvorslin', 'harina', 'helleveeg', 'henriette', 'herbergierster', 'herinne', 'hermine', 'hertogin-weduwe',
 'hofdames', 'hoofdverpleegster', 'huismoeder', 'intrigante', 'iroelstra', 'jeunesse', 'jongejuffr', 'jongejuffrouwen',
 'jonkvrouwe', 'judith', 'juffertje', 'juffr', 'jufvrouw', 'kamenier', 'keetje', 'keizerin-moeder', 'keizerin-weduwe',
 'kemp', 'kindermeid', 'kindermoordenares', 'kleindochter', 'konidgin', 'koningin-moeder', 'koningin-regentes',
 'koningsdochter', 'koppelaarster', 'kroonprinses', 'kruisnet', 'kunstenaarshand', 'kunstenares', 'kunstenaresse', 'landloopster',
 'leerare', 'leerarea', 'leermeesteresse', 'leidster', 'leugenaarster', 'lezeressen', 'liesbeth', 'lijderes', 'linnennaaister',
 'madame', 'mademoiselle', 'maitresse', 'marketenster', 'martelares', 'meesteresse', 'mejuffrouw', 'melodieuze',
 'mevronw', 'mevrouwtje', 'misdadigster', 'mlle', 'mme', 'modemaakster', 'modiste', 'moedermaagd', 'moedor', 'moordenares',
 'muze', 'negerin', 'onderwijzeres', 'onderwyzeres', 'opvoedster', 'opvolgster', 'paardrijdster', 'pianiste', 'pleegdochter',
 'pleegouders', 'pleegzuster', 'pompadour', 'portheine', 'priesteres', 'prostituees', 'sara', 'schoonmama', 'schoonouders',
 'schrijfster', 'socialiste', 'sultanes', 'susanna', 'toovenares', 'tragedienne', 'troela', 'troelatra', 'vischvrouwen',
 'voedsters', 'voogdesse', 'voorgangsters', 'voorvechtster', 'vrijsters', 'vroedvrouwen', 'vronw', 'vrouwenbeweging', 'vrouwengestalte',
 'vrouwenrechten', 'vrouwspersonen', 'waschvrouw', 'wednwe', 'weduwo', 'werkmeid', 'wijf', 'winkelierster',
 'winkeljuffrouw', 'ziekenverpleegster', 'zigeunerin', 'zusje', 'zusteren'})

In [30]:
male_words = male_words.union({'acrobaat', 'acteur', 'ambachtsman', 'anton', 'apostel', 'arbeider', 'arbeidsman', 'aristocraat', 'armenier', 'augustinus', 'bakkersknecht',
 'bengel', 'berichtgever', 'bevorderaar', 'bokser', 'bondgenoot', 'bondspenningmeester', 'bootsman', 'borgcsius',
 'borgesiua', 'borgesius', 'botman', 'briefschrijver', 'brigadier-majoor', 'broeder', 'broer', 'broodbakker',
 'bruigom', 'buurman', 'carlos', 'chirurgijn', 'chirurgijn-majoor', 'chrysostomus', 'denker', 'diaken', 'diplomaat',
 'dwerg', 'filosoof', 'franciskaner', 'freetrader', 'freiherr', 'fritz', 'gendarm', 'gendarmen', 'gentleman', 'gerrit', 'gierigaard', 'grjjsaard', 'groot-meester', 'grootvader',
 'grysaard', 'handwerker', 'handwerksman', 'huisknecht', 'huismeester', 'janmaat', 'johannessen', 'jongeling',
 'jongeman', 'jongen', 'jonggezel', 'jongman', 'kamerheer', 'kapiteiu', 'kees', 'keizer', 'kerel', 'kerkvader',
 'kleinzoon', 'kloosterling', 'knaap', 'knaapje', 'knecht', 'knechts', 'kneoht', 'kolonel', 'kolos', 'kolossus',
 'koniug', 'koopman', 'koopvaardij-kapitein', 'kroonpretendent', 'kroonprins', 'kruidenier', 'landbouwer', 'landlooper',
 'leenheer', 'luit-kolonel', 'luitenant', 'luitenant-kolonel', 'majoor', 'manufacturier', 'matroos', 'mau', 'meesterknecht', 'melkboer',
 'neef', 'oorrespondent', 'oppassers', 'opperstuurman', 'oud-oom', 'overgrootvader', 'patricier', 'patroon', 'philosoof',
 'ploegbaas', 'politicus', 'politiebeambte', 'president-kerkvoogd', 'priester', 'prijswinner', 'rentenier', 'rijksveldwachter',
 'rynders', 'saletjonker', 'scheepsjongen', 'scheepskok', 'schelm', 'schooier', 'schoonvader', 'schoonzoon',
 'sinjeur', 'slagersjongen', 'snaak', 'snuiter', 'sonneman', 'spekslager', 'staatsdienaar',
 'stakkert', 'stumper', 'tooneeldirecteur', 'tooneelkunstenaar', 'toovenaar', 'torero', 'tremmer', 'troonpretendent', 'vader', 'vagebond', 'vasal', 'vazal', 'veehouder', 'vent', 'verleider', 'verloofde', 'vischboer',
 'visscher', 'visschersman', 'visseher', 'voerman', 'vriendt', 'vrijgezel', 'vrijmetselaren', 'vrjjer', 'vryer', 'vryheer', 'vuilnisman',
 'wachtmeester', 'weduwenaar', 'weikman', 'werner', 'werold', 'wierenga', 'wijsgeer', 'willeman', 'winnaar',
 'woekeraar', 'woesteling', 'worstelaar', 'ziekenoppasser', 'ziekenvader', 'ziekenverpleger', 'zwager'})

In [31]:
len(female_words)

524

In [32]:
FACET = "Politek" # "Politek"
FACET_VALUES = df_meta[FACET].unique()
FACET_VALUES

array(['Neutraal', 'Liberaal', nan, 'Sociaal-democratisch', 'Katholiek',
       'Protestant', 'Conservatief'], dtype=object)

# Timelines

In [33]:
SELECTED_FACETS = ['Neutraal', 'Liberaal','Sociaal-democratisch', 'Katholiek','Protestant']

In [34]:
katholiek_models = select_model_by_facet_value('Katholiek')
liberaal_models = select_model_by_facet_value('Liberaal')
protestant_models = select_model_by_facet_value('Protestant')
years = sorted(set(katholiek_models).intersection(set(liberaal_models)).intersection(set(protestant_models)))

In [35]:
years

[1840, 1845, 1850, 1855, 1860, 1865, 1870, 1875, 1880, 1885, 1890, 1895]

In [36]:
from gensim.models.word2vec import Word2Vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

base_model = "/kbdata/Processed/Models/BaseModel-1800-1909.w2v.model"
base_model = Word2Vec.load(base_model)
print(base_model)


2020-10-27 19:02:46,032 : INFO : loading Word2Vec object from /kbdata/Processed/Models/BaseModel-1800-1909.w2v.model
2020-10-27 19:02:47,413 : INFO : loading wv recursively from /kbdata/Processed/Models/BaseModel-1800-1909.w2v.model.wv.* with mmap=None
2020-10-27 19:02:47,414 : INFO : loading vectors from /kbdata/Processed/Models/BaseModel-1800-1909.w2v.model.wv.vectors.npy with mmap=None
2020-10-27 19:03:18,238 : INFO : setting ignored attribute vectors_norm to None
2020-10-27 19:03:18,240 : INFO : loading vocabulary recursively from /kbdata/Processed/Models/BaseModel-1800-1909.w2v.model.vocabulary.* with mmap=None
2020-10-27 19:03:18,240 : INFO : loading trainables recursively from /kbdata/Processed/Models/BaseModel-1800-1909.w2v.model.trainables.* with mmap=None
2020-10-27 19:03:18,241 : INFO : loading syn1neg from /kbdata/Processed/Models/BaseModel-1800-1909.w2v.model.trainables.syn1neg.npy with mmap=None
2020-10-27 19:03:48,484 : INFO : setting ignored attribute cum_table to None


Word2Vec(vocab=414769, size=300, alpha=0.025)


In [37]:
base_model_early = "/kbdata/Processed/Models/BaseModel-1800-1870.w2v.model"
model_early = Word2Vec.load(base_model_early)
print(model_early)

2020-10-27 19:03:49,345 : INFO : loading Word2Vec object from /kbdata/Processed/Models/BaseModel-1800-1870.w2v.model
2020-10-27 19:03:50,181 : INFO : loading wv recursively from /kbdata/Processed/Models/BaseModel-1800-1870.w2v.model.wv.* with mmap=None
2020-10-27 19:03:50,182 : INFO : loading vectors from /kbdata/Processed/Models/BaseModel-1800-1870.w2v.model.wv.vectors.npy with mmap=None
2020-10-27 19:04:01,747 : INFO : setting ignored attribute vectors_norm to None
2020-10-27 19:04:01,748 : INFO : loading vocabulary recursively from /kbdata/Processed/Models/BaseModel-1800-1870.w2v.model.vocabulary.* with mmap=None
2020-10-27 19:04:01,748 : INFO : loading trainables recursively from /kbdata/Processed/Models/BaseModel-1800-1870.w2v.model.trainables.* with mmap=None
2020-10-27 19:04:01,749 : INFO : loading syn1neg from /kbdata/Processed/Models/BaseModel-1800-1870.w2v.model.trainables.syn1neg.npy with mmap=None
2020-10-27 19:04:13,371 : INFO : setting ignored attribute cum_table to None


Word2Vec(vocab=155879, size=300, alpha=0.025)


In [38]:
model_early.most_similar('schaamte',topn=20)

/home/kaspar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2020-10-27 19:04:13,705 : INFO : precomputing L2-norms of word weight vectors


[('wroeging', 0.6259299516677856),
 ('laagheid', 0.623017430305481),
 ('medelijden', 0.5965837836265564),
 ('walging', 0.5864320397377014),
 ('zelfzucht', 0.5812700986862183),
 ('huichelarij', 0.5809855461120605),
 ('boosheid', 0.5807294249534607),
 ('mededoogen', 0.5723110437393188),
 ('verwaandheid', 0.5714962482452393),
 ('laffe', 0.5670630931854248),
 ('hoogmoed', 0.5643799901008606),
 ('afschuw', 0.5642658472061157),
 ('blozen', 0.561642587184906),
 ('verachtelijke', 0.5611639022827148),
 ('spotternij', 0.5594935417175293),
 ('ijdelheid', 0.5563795566558838),
 ('schande', 0.5539793968200684),
 ('veinzerij', 0.5473310947418213),
 ('verblinding', 0.5450595021247864),
 ('wellust', 0.5419415831565857)]

In [39]:
# female_words_new = set()
# for w in female_words:
#     for m in [model,model_early]:
#         if m.wv.__contains__(w):
#             female_words_new.update(set([k for k,v in m.wv.most_similar(w,topn=3)]))
        

In [40]:
# female_words_new = {w for w in female_words_new if not w in female_words}

In [ ]:
# male_words_new = set()
# for w in male_words:
#     for m in [model,model_early]:
#         if m.wv.__contains__(w):
#             male_words_new.update(set([k for k,v in m.wv.most_similar(w,topn=3)]))
        

In [ ]:
# male_words_new = {w for w in male_words_new if not w in male_words}

In [ ]:
base_model.most_similar('emotie',topn=20)

In [ ]:
model_early.most_similar('emotie',topn=20)

In [ ]:
target_list = ['kind',"kiud",'kinderen','kindje','kiud','zuigeling','kiad']
# target_list = ['wellust','wellustige',"wellustig",'hartstocht',
#                "wulpschheid",'hartstogt','begeerten','verlangen',
#                "zingenot","smachtende","dartele",'driften','hartstochten']
# target_list = ["huisselijk","huiselgk","huiselyk","gezellig","familieleven"]
target_list = ["emotie","ontroering", "gemoedsbeweging", "opgewondenheid", "agitatie", 
                "aandoening", "gemoedsaandoening", "sensatie", "fenfatie",
               "emoties","ontroeringen","opwinding"]


In [ ]:
#female_words = ['moeder',"moeders","moederschap","huismoeders","huismoeder"]
#male_words = ["vader","vaders","vaderschap","huisvader",'huisvaders']
#female_words = ["vrouw","vrouwen",]
#male_words = ["man","mannen","lieden"]

In [ ]:
results = {}
finetuned = False
if finetuned:
    base_bias = compute_bias(female_words,male_words,target_list,base_model)
else:
    base_bias = .0
    
for year in tqdm(years):
    lib_model = Word2Vec.load(str(liberaal_models[year]))
    lib_bias = compute_bias(female_words,male_words,target_list,lib_model) + base_bias
    kath_model = Word2Vec.load(str(katholiek_models[year]))
    kath_bias = compute_bias(female_words,male_words,target_list,kath_model) + base_bias
    protestant_model = Word2Vec.load(str(protestant_models[year]))
    protestant_bias = compute_bias(female_words,male_words,target_list,protestant_model) + base_bias
    results[year] = (lib_bias,kath_bias,protestant_bias)

In [ ]:
df_result = pd.DataFrame(results).T#.plot()
df_result.columns = ["Liberaal", "Katholiek", "Protestant"]
df_result.loc[1850:].plot()

In [ ]:
df_result["Average"] = df_result.mean(axis=1)
df_result["Average"].loc[1850:].plot()

# Inspect Scores

In [ ]:
def inspect_bias(p1,p2,target,model,metric=cosine_sim):
    p1 = [p for p in p1 if  model.wv.__contains__(p)]
    p2 = [p for p in p2 if  model.wv.__contains__(p)]
    target = [t for t in target if  model.wv.__contains__(t)]
    p1_scores, p2_scores = defaultdict(float),defaultdict(float)
    for t in target:
        for p in p1:
            p1_scores[(p,target[0])] += metric(model.wv.__getitem__(p),model.wv.__getitem__(t))
        for p in p2:
            p2_scores[(p,target[0])] += metric(model.wv.__getitem__(p),model.wv.__getitem__(t))
    return (p1_scores,p2_scores)

In [ ]:
#for year in tqdm(years):
YEAR = 1850
lib_model = Word2Vec.load(str(liberaal_models[YEAR]))
lib_bias = inspect_bias(female_words,male_words,target_list,lib_model)
kath_model = Word2Vec.load(str(katholiek_models[YEAR]))
kath_bias = inspect_bias(female_words,male_words,target_list,kath_model)
protestant_model = Word2Vec.load(str(protestant_models[YEAR]))
protestant_bias = inspect_bias(female_words,male_words,target_list,protestant_model)

In [ ]:
pd.Series(list(lib_bias[0].values())).plot(kind='density')
pd.Series(list(lib_bias[1].values())).plot(kind='density')

In [ ]:
pd.Series(list(kath_bias[0].values())).plot(kind='density')
pd.Series(list(lib_bias[0].values())).plot(kind='density')

In [ ]:
pd.Series(list(protestant_bias[0].values())).plot(kind='density')
pd.Series(list(protestant_bias[1].values())).plot(kind='density')

In [ ]:
sorted(lib_bias[0].items(),key=lambda x: x[1],reverse=True)[:10]

In [ ]:
sorted(kath_bias[1].items(),key=lambda x: x[1],reverse=True)[:10]

In [ ]:
sorted(kath_bias[0].items(),key=lambda x: x[1],reverse=True)[:10]

In [ ]:
sorted(lib_bias[0].items(),key=lambda x: x[1],reverse=True)[:10]

# Barplots

In [ ]:
FACET = "Provincie" # "Politek" | Provincie
FACET_VALUES = df_meta[FACET].unique()
FACET_VALUES

In [ ]:
YEAR = 1880

selected_models = {}

for facet in FACET_VALUES:
    model_paths = select_model_by_facet_value(facet)
    
    selected_models[facet] = model_paths.get(YEAR,None)
    
selected_models = {w:v for w,v in selected_models.items() if v}

In [ ]:
selected_models

In [ ]:
results = {}
for w,v in tqdm(selected_models.items()):
    model = Word2Vec.load(str(v))
    results[w] = compute_bias(female_words,male_words,target_list,model)

In [ ]:
pd.Series(results).plot(kind='bar')

## To reproduce results

In [ ]:
#female_words = ['moeder',"moeders","moederschap","huismoeders","huismoeder"]; male_words = ["vader","vaders","vaderschap"]
# method = compute_bias_average_vector